In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
path = Path('/home/jupyter/fpl-prediction/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
77285,Wilfried_Zaha,17,3,90,Crystal Palace,Brighton and Hove Albion,0.464142,0.453089,True,8,1920,1.0
77286,Michael_Keane,17,2,90,Everton,Manchester United,1.033814,1.560524,False,2,1920,1.0
77287,Jairo_Riedewald,17,3,45,Crystal Palace,Brighton and Hove Albion,0.464142,0.453089,True,1,1920,1.0
77288,Aaron_Mooy,17,3,90,Brighton and Hove Albion,Crystal Palace,0.453089,0.464142,False,2,1920,1.0
77289,John_Ruddy,17,1,0,Wolverhampton Wanderers,Tottenham Hotspur,0.746215,2.001099,True,0,1920,1.0


In [5]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

18

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [7]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
0,Ederson_Santana de Moraes,18,1,90,Manchester City,Leicester City,2.711234,1.054253,True,2,1920
1,Hugo_Lloris,18,1,0,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,0,1920
2,Cenk_Tosun,18,4,68,Everton,Arsenal,1.033814,1.413491,True,2,1920
3,Callum_Robinson,18,3,0,Sheffield United,Brighton and Hove Albion,0.242030,0.453089,False,0,1920
4,Heung-Min_Son,18,3,61,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,-1,1920
5,João Filipe Iria_Santos Moutinho,18,3,90,Wolverhampton Wanderers,Norwich,0.746215,0.295212,False,4,1920
6,Danny_Rose,18,2,16,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,1,1920
7,Martin_Kelly,18,2,90,Crystal Palace,Newcastle United,0.464142,0.561705,False,2,1920
8,Harry_Winks,18,3,0,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,0,1920
9,Scott_Carson,18,1,0,Manchester City,Leicester City,2.711234,1.054253,True,0,1920


In [8]:
df_latest.shape

(520, 11)

In [9]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [10]:
last_play_proba.head()

,player,play_proba
0,Ahmed_El Mohamady,1.0
1,James_Chester,1.0
2,Neil_Taylor,1.0
3,Kortney_Hause,1.0
4,Jonathan_Kodjia,1.0


In [11]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [12]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
0,Ederson_Santana de Moraes,18,1,90,Manchester City,Leicester City,2.711234,1.054253,True,2,1920,1.00
1,Hugo_Lloris,18,1,0,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,0,1920,0.00
2,Cenk_Tosun,18,4,68,Everton,Arsenal,1.033814,1.413491,True,2,1920,1.00
3,Callum_Robinson,18,3,0,Sheffield United,Brighton and Hove Albion,0.242030,0.453089,False,0,1920,0.75
4,Heung-Min_Son,18,3,61,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,-1,1920,1.00
5,João Filipe Iria_Santos Moutinho,18,3,90,Wolverhampton Wanderers,Norwich,0.746215,0.295212,False,4,1920,1.00
6,Danny_Rose,18,2,16,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,1,1920,1.00
7,Martin_Kelly,18,2,90,Crystal Palace,Newcastle United,0.464142,0.561705,False,2,1920,1.00
8,Harry_Winks,18,3,0,Tottenham Hotspur,Chelsea,2.001099,1.757609,True,0,1920,1.00
9,Scott_Carson,18,1,0,Manchester City,Leicester City,2.711234,1.054253,True,0,1920,1.00


In [13]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [14]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
77805,Grant_Hanley,18,2,90,Norwich,Wolverhampton Wanderers,0.295212,0.746215,True,0,1920,0.50
77806,Roberto_Pereyra,18,3,13,Watford,Manchester United,0.433797,1.560524,True,1,1920,0.75
77807,Sead_Kolasinac,18,2,0,Arsenal,Everton,1.413491,1.033814,False,0,1920,0.75
77808,Stuart_Armstrong,18,3,75,Southampton,Aston Villa,0.469252,0.522434,False,2,1920,1.00
77809,Rúben Diogo_da Silva Neves,18,3,90,Wolverhampton Wanderers,Norwich,0.746215,0.295212,False,2,1920,1.00


In [15]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [16]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [17]:
fixtures.head(10)

,home_team,away_team,gw
179,West Ham United,Liverpool,34
180,Aston Villa,Norwich,19
181,Bournemouth,Arsenal,19
182,Chelsea,Southampton,19
183,Crystal Palace,West Ham United,19
184,Everton,Burnley,19
185,Leicester City,Liverpool,19
186,Manchester United,Newcastle United,19
187,Sheffield United,Watford,19
188,Tottenham Hotspur,Brighton and Hove Albion,19


In [18]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [19]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,34,West Ham United,Liverpool,21,14
1,19,Aston Villa,Norwich,7,45
2,19,Bournemouth,Arsenal,91,3
3,19,Chelsea,Southampton,8,20
4,19,Crystal Palace,West Ham United,31,21


In [20]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [21]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [22]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,34,Liverpool,West Ham United,Andrew_Robertson,2,70,100,False
1,34,Liverpool,West Ham United,Trent_Alexander-Arnold,2,72,100,False
2,34,Liverpool,West Ham United,Virgil_van Dijk,2,64,100,False
3,34,Liverpool,West Ham United,Joseph_Gomez,2,51,100,False
4,34,Liverpool,West Ham United,Joel_Matip,2,52,0,False


In [23]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,34,West Ham United,Liverpool,Aaron_Cresswell,2,47,100,True
1,34,West Ham United,Liverpool,Angelo_Ogbonna,2,45,None,True
2,34,West Ham United,Liverpool,Arthur_Masuaku,2,43,100,True
3,34,West Ham United,Liverpool,Pablo_Zabaleta,2,42,None,True
4,34,West Ham United,Liverpool,Ryan_Fredericks,2,45,0,True


In [24]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 37.5,
 'Aaron_Cresswell': 76.66666666666667,
 'Aaron_Lennon': 29.4,
 'Aaron_Mooy': 90.0,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abdoulaye_Doucouré': 82.2,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 30.666666666666668,
 'Adam_Masina': 67.5,
 'Adam_Smith': 0.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 89.6,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 72.0,
 'Adrián_San Miguel del Castillo': 25.5,
 'Ahmed_El Mohamady': 36.0,
 'Ainsley_Maitland-Niles': 54.0,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 2.5,
 'Alex_Iwobi': 71.4,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 45.75,
 'Alexander_Tettey': 66.5,
 'Alexandre_Lacazette': 35.6,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 4.4,
 'Alisson_Ramses Becker': 85.0,
 'Allan_Saint-Maximin': 79.0,
 'Andre_Gray': 10.8,
 'Andreas_Christensen': 36.0,
 'Andreas_Pereira': 23.2,
 'Andrew_Robertson': 71.5,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 15.666666666666666,
 'Andros_Townsend': 46.25,
 'An

In [25]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [26]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,34,West Ham United,Liverpool,Aaron_Cresswell,2,4.7,1.00,True,0.699185,2.460966,1920,76.666667
1,34,West Ham United,Liverpool,Angelo_Ogbonna,2,4.5,1.00,True,0.699185,2.460966,1920,90.000000
2,34,West Ham United,Liverpool,Arthur_Masuaku,2,4.3,1.00,True,0.699185,2.460966,1920,11.500000
3,34,West Ham United,Liverpool,Pablo_Zabaleta,2,4.2,1.00,True,0.699185,2.460966,1920,0.000000
4,34,West Ham United,Liverpool,Ryan_Fredericks,2,4.5,0.00,True,0.699185,2.460966,1920,0.000000
5,34,West Ham United,Liverpool,Issa_Diop,2,4.5,1.00,True,0.699185,2.460966,1920,0.666667
6,34,West Ham United,Liverpool,Fabián_Balbuena,2,4.3,1.00,True,0.699185,2.460966,1920,90.000000
7,34,West Ham United,Liverpool,Winston_Reid,2,3.9,1.00,True,0.699185,2.460966,1920,0.000000
8,34,West Ham United,Liverpool,Ben_Johnson,2,4.0,1.00,True,0.699185,2.460966,1920,0.000000
9,34,West Ham United,Liverpool,Marko_Arnautovic,4,7.0,0.00,True,0.699185,2.460966,1920,0.000000


In [27]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')